In [4]:
%pip install prefect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55

## Prefect Cloud Login Required

Before running the cells below, please log in to Prefect Cloud.  
Use the same parameters as in the **ot2_orchestrator_prefect** notebook.

---

### 1. Get Your API Key
1. Go to [Prefect Cloud](https://app.prefect.cloud).
2. Click the workspace name in the top-left (e.g., `default`).
3. Select **API Keys** from the menu.
4. Click **+** to create a new key.
5. Copy the key — you will add it to Colab Secrets later.

---

### 2. Get Your API URL
1. Open your workspace in the browser.
2. Copy the URL from the address bar — it may look like:  
   `https://app.prefect.cloud/account/.../workspace/.../(extra path after workspace)`
3. Convert it to the API format — replace the **two ellipses** (`...`) above with the corresponding values from your URL, and **remove anything after the second ellipsis**:  
   `https://api.prefect.cloud/api/accounts/.../workspaces/...`  
   *(Example: `https://api.prefect.cloud/api/accounts/12345678-abcd-1234-abcd-1234567890ab/workspaces/abcdef12-3456-7890-abcd-abcdef123456`)*

---

### 3. Set Up Colab Secrets
1. Open the **left sidebar** in Colab.
2. Click the **key icon** labeled **Secrets**.
3. Click **+ Add new secret** and create:
   - **Name:** `PREFECT_API_KEY` → **Value:** your API key.
   - **Name:** `PREFECT_API_URL` → **Value:** your API URL.
4. Save each secret after entering the value.
5. Run the following cell to authenticate.

In [5]:
# @title Prefect Cloud Authentication (Colab Secrets)
import os
from google.colab import userdata

PREFECT_API_KEY = userdata.get("PREFECT_API_KEY")
PREFECT_API_URL = userdata.get("PREFECT_API_URL")

if not PREFECT_API_KEY:
    raise ValueError(
        "Missing secret PREFECT_API_KEY. Please add it in the Colab 'Secrets' sidebar and re-run this cell."
    )
if not PREFECT_API_URL:
    raise ValueError(
        "Missing secret PREFECT_API_URL. Please add it in the Colab 'Secrets' sidebar and re-run this cell."
    )

os.environ["PREFECT_API_KEY"] = PREFECT_API_KEY
os.environ["PREFECT_API_URL"] = PREFECT_API_URL

!prefect deployment ls

                                  Deployments                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Name                          ┃ ID                           ┃ Work Pool     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mix-color/mix-color           │ dc972dfb-2cdc-411b-8096-ec1… │               │
│ move-sensor-back/move-sensor… │ 9835a1f2-6b19-4837-86fa-fcb… │               │
│ move-sensor-to-measurement-p… │ ad982977-d6fb-412a-b14b-513… │               │
└───────────────────────────────┴──────────────────────────────┴───────────────┘


Checkout the prefect_serve_standalone notebook for output

In [7]:
from prefect.settings import temporary_settings, PREFECT_API_KEY, PREFECT_API_URL
from prefect.deployments import run_deployment

with temporary_settings(
    {
        PREFECT_API_KEY: os.environ["PREFECT_API_KEY"],
        PREFECT_API_URL: os.environ["PREFECT_API_URL"]
    }
):

  await run_deployment(
      name="mix-color/mix-color",
      parameters={"R": 120, "Y": 50, "B": 80, "mix_well": "B2"},
  )

  await run_deployment(
      name="move-sensor-to-measurement-position/move-sensor-to-measurement-position",
      parameters={"mix_well": "B2"},
  )

  await run_deployment(name="move-sensor-back/move-sensor-back")